# On homomorphisms

This section is devoted to understand how can we compute on the ciphertext sums and multiplications. At the end of it we will have an idea on what is fully homomorphic encryption.

In [1]:
from crypt import PaillierKeyGenerator, PaillierEncrypt, PaillierDecrypt
from crypt import RSAKeyGenerator, RSAEncrypt, RSADecrypt
from crypt import ElGamalKeyGenerator, ElGamalEncrypt, ElGamalDecrypt

from random import randrange, seed

In [2]:
bits = 64
seed(1)

PublicKeyGamal, PrivateKeyGamal = ElGamalKeyGenerator(bits)
PublicKeyPaillier, PrivateKeyPaillier = PaillierKeyGenerator(bits)
PublicKeyRSA, PrivateKeyRSA = RSAKeyGenerator(bits)

m1, m2, m3 = randrange(2, 2**(bits//2)), randrange(2, 2**(bits//2)), randrange(2, 2**(bits//2))